In [1]:
import pandas as pd

# 전체 계정의 특징 분류
* ant_id : auto 계정 중 재화수집 계층
* master_id : auto 계정 중 관리자 계층
* auto_id : ant_id와 master_id의 합
* normal_id : 일반 계정
* hard_id : hard play 계정
* human_id : normal_id와 master_id의 합
* 8610_id : 튜토리얼 계정
* jump_id : 고속성장 계정

In [2]:
tr = pd.read_csv('../3.통합/4.주당평균_파생변수/train_create_mean.csv')
te = pd.read_csv('../3.통합/4.주당평균_파생변수/test_create_mean.csv')
to = pd.concat([tr, te], sort=False)

auto = pd.read_csv('auto_list.csv')
hard = pd.read_csv('hard_list_fin.csv')
dummy = pd.read_csv('8610_list.csv')
jump = pd.read_csv('jump_list.csv')

In [3]:
total_list = pd.merge(auto, hard, how='outer').merge(dummy, how='outer').merge(jump, how='outer')
total_list = total_list.sort_values(by='acc_id').reset_index(drop=True)

In [4]:
normal = pd.DataFrame(set(to['acc_id'])-set(total_list['acc_id'])).rename(columns={0: 'acc_id'})
normal['normal_id'] = 1

total_list = pd.merge(total_list, normal, how='outer')

In [5]:
human = pd.DataFrame(total_list[
    (total_list['normal_id']==1)
    | (total_list['hard_id']==1)
]['acc_id']).rename(columns={0: 'acc_id'}).reset_index(drop=True)
human['human_id'] = 1

total_list = pd.merge(total_list, human, how='outer').sort_values(by='acc_id').reset_index(drop=True).fillna(0)
total_list = total_list[['acc_id', 'ant_id', 'master_id', 'auto_id', 'normal_id', 'hard_id', 'human_id', '8610_id', 'jump_id']]
total_list[total_list.columns[1:]] = total_list[total_list.columns[1:]].astype(int)

In [6]:
total_list.head()

,acc_id,ant_id,master_id,auto_id,normal_id,hard_id,human_id,8610_id,jump_id
0,0000264b01392acfde44f9d8494f112a701dc5d3e5fda6...,1,0,1,0,0,0,0,0
1,0001f130e89288ff0df167b998f6eb7085687f411fcb72...,0,0,0,1,0,1,0,0
2,00028f0ad71c07f48aef465fd0c98ba6e3f0f3de3d2f7d...,0,0,0,0,0,0,1,0
3,0002a56a036206aea3a6c6ebc985df4a2d1987b562e7f0...,0,0,0,0,0,0,0,1
4,00036e5b6a197c196fa10fc0ad4e2853b22294dff64d27...,1,0,1,0,0,0,0,0


In [7]:
total_list.to_csv('total_list.csv',index=False)